In [ ]:
import projectq
import math
from math import sqrt
from projectq import MainEngine
from projectq.ops import H, Measure, X, MatrixGate, SqrtXGate, CNOT,All,C, Toffoli

from cmath import exp
from cmath import pi

In [2]:
eng = MainEngine()

In [3]:
n = 5



In [4]:
e_pos = exp(1j*pi/3)
e_neg = exp(-1j*pi/3)

e_neg_2 = exp(-1j*pi/6)
e_pos_2 = exp(1j*pi/6)

V_3 = MatrixGate([[1/sqrt(3), 0, 1/sqrt(3), e_pos/sqrt(3)],
                 [1/sqrt(3), 0, -1/sqrt(3)*e_neg, e_pos/sqrt(3)],
                 [1/sqrt(3), 0, e_neg_2*e_pos.imag/(1j*sqrt(3)*e_pos_2.real), -1/sqrt(3)],
                 [0,1,0,0]])

U_2 = MatrixGate([[(1/sqrt(2)),-(1j/sqrt(2))],[-(1j/sqrt(2)),(1/sqrt(2))]])

In [5]:
eligTUP = [[1,1],[2,1],[3,1], [4,1],[5,1]]

In [6]:

def Consistency5P(X0, Y,eligTUP):
    print("Cons 5P")
    #here we know that we have 5 eligible parties
    C(C(C(Toffoli))) |(X0[0],X0[1],X0[2],X0[3],X0[4],Y[0])
    All(X)| X0
    C(C(C(Toffoli))) |(X0[0],X0[1],X0[2],X0[3],X0[4],Y[0])
    All(X)| X0
   

    
    
    Measure | Y[0]
    
    eng.flush()
    print("consistencyval", format(int(Y[0])))
    return(format(int(Y[0])))


def Consistency4P(X0, Y,eligTUP):
    print("Cons 4P")
    
    conseliglist4 = []
    for i in range(len(X0)):
        if (eligTUP[i][1] == 1):
            conseliglist4.append(i)
    print("eliglist", conseliglist4)
    #here we know that we have 4 eligible parties
    C(C(Toffoli)) | (X0[conseliglist4[0]], X0[conseliglist4[1]], X0[conseliglist4[2]], X0[conseliglist4[3]],  Y[0])
    All(X) | X0
    C(C(Toffoli)) | (X0[conseliglist4[0]],X0[conseliglist4[1]],X0[conseliglist4[2]],X0[conseliglist4[3]], Y[0])
    All(X) | X0

    
    Measure | Y[0]
    eng.flush()
    print("cons val", format(int(Y[0])))
    return(format(int(Y[0])))


def Consistency3P(X0,Y, eligTUP):
    print("Cons 3P")
    
    conseliglist3 = []
    for i in range(len(X0)):
        if (eligTUP[i][1] == 1):
            conseliglist3.append(i)
    print("eliglist", conseliglist3)
    
    #here we know that we have 3 eligible parties
    C(Toffoli) | (X0[conseliglist3[0]],X0[conseliglist3[1]],X0[conseliglist3[2]], Y[0])
    All(X) | X0
    C(Toffoli) | (X0[conseliglist3[0]],X0[conseliglist3[1]],X0[conseliglist3[2]], Y[0])
    All(X) | X0

 
    
    Measure | Y[0]
    eng.flush()
    print("cons val 3P", format(int(Y[0])))
    
    return(format(int(Y[0])))


def Consistency2P(X0, Y, eligTUP):
    print("Cons 2P")
    
    conseliglist2 = []
    for i in range(len(X0)):
        if (eligTUP[i][1] == 1):
            conseliglist2.append(i)
    print("conseliglist", conseliglist2)
    
    #here we know that we have 2 eligible parties
    (Toffoli) | (X0[conseliglist2[0]],X0[conseliglist2[1]], Y[0])
    All(X) | X0
    (Toffoli) | (X0[conseliglist2[0]],X0[conseliglist2[1]], Y[0])
    All(X) | X0
    
    
    Measure | Y[0]

    eng.flush()
    
    print("cons val 2P", format(int(Y[0])))
    
    return(format(int(Y[0])))

In [7]:
def Consistency(X0,Y,eligTUP):
    NUM_EP = 0
    for i in range(len(eligTUP)):
        if (eligTUP[i][1] == 1):
            NUM_EP = NUM_EP + 1
    if (NUM_EP == 5):
        return(Consistency5P(X0,Y, eligTUP))
    if (NUM_EP == 4):
        return(Consistency4P(X0,Y, eligTUP))
    if (NUM_EP == 3):
        return(Consistency3P(X0,Y, eligTUP))
    if (NUM_EP == 2):
        return(Consistency2P(X0,Y, eligTUP))
    

In [8]:
def BreakSymmetry_VK(X0,X1,Y, ph,eligTUP):
    
    X | Y[0]
    print("odd break symmetry")
    for i in range(len(X0)):
        if (eligTUP[i][1] == 1):
            print("i CNOT", i)
            CNOT | (X0[i],X1[i])
    
    e_pos_ = exp(1j*pi/ph)
    e_neg_ = exp(1j*pi/ph)
    

    e_pos_2 = exp(1j*pi/(2*ph))
    e_neg_2 = exp(1j*pi/(2*ph))
    

    Rk = e_pos.real
    Ik = e_pos.imag*1j
    R2k = e_pos_2.real
    
    print("Rk", Rk)
    
    NF = 1/sqrt(Rk + 1)

    V_k = MatrixGate([[(1/sqrt(2))*NF, 0, (1/sqrt(2))*NF, (e_pos/sqrt(2))*NF],
                      [(1/sqrt(2))*NF, 0, (-sqrt(Rk)*e_neg)*NF, (e_pos/sqrt(2))*NF],
                      [sqrt(Rk)*NF, 0, (e_neg_2*Ik/(1j*sqrt(2)*R2k))*NF, (-sqrt(Rk))*NF],
                      [0,(sqrt(Rk+1))*NF,0,0]]) 
    

    for i in range(len(X0)):
        if (eligTUP[i][1] == 1):
            V_k | (X1[i], X0[i])
    return()
            
def BreakSymmetry_UK(X0,X1,Y,ph,eligTUP):
    
    X | Y[0]
    print("even breaksymmetry")
    
    e_pos_g = exp(1j*pi/ph)
    e_neg_g = exp(1j*pi/ph)
    
    U_k = MatrixGate([[1/sqrt(2), e_neg_g/sqrt(2)],
                      [-e_pos_g/sqrt(2), 1/sqrt(2)]])
    
    for i in range(len(X0)):
        if (eligTUP[i][1] == 1):
            print("BS eligtup", eligTUP[i][0])
            U_k | X0[i]
    return()

In [9]:
def Find_Zvals(X0, X1, eligTUP):
    print("in findz_vals")
    All(Measure) | X0
    All(Measure) | X1
    
    zval_list = []
    eng.flush()
    
    for i in range(len(X0)):
        print("eligtup[i][1]",eligTUP[i][1])
        if (eligTUP[i][1] == 1):
            print("eligtup ga z val calc", eligTUP[i][0])
            if (format(int(X0[i])) == '0'):
                z0 = 0
                print("X0, i", format(int(X0[i])),i)
            if (format(int(X0[i])) == '1'):
                z0 = 2
                print("X0, i",format(int(X0[i])),i)
            if (format(int(X1[i])) == '0'):
                z1 = 0
                print("X1, i", format(int(X1[i])),i)
            if (format(int(X1[i])) == '1'):
                z1 = 1
                print("X1, i", format(int(X1[i])),i)
            zval_list.append([i,z0+z1])
        if (eligTUP[i][1] == 0):
            zval_list.append([i, -1])
    print("zval_list",zval_list)
    return(zval_list)



In [10]:
def new_elig_parties_max(X0, X1, eligTUP):
    zvallist = Find_Zvals(X0, X1, eligTUP)
    maxz = max(zvallist[0][1], zvallist[1][1], zvallist[2][1], zvallist[3][1], zvallist[4][1])
    
    new_elig_parties = []
    for i in range(len(X0)):
        if (zvallist[i][1] == maxz):
            new_elig_parties.append(zvallist[i][0])
        else:
            zvallist[i][1] = -1
    print("new_elig_parties", new_elig_parties)
    return(new_elig_parties)





In [11]:

def LE(n,eligTUP,phase):
    X0 = eng.allocate_qureg(n)
    Y = eng.allocate_qureg(1)
    X1 = eng.allocate_qureg(n)
    
    print("phase",phase)
    

    for i in range(len(X0)):
        if (eligTUP[i][1] == 1):
            #print("H, i", i)
            H | X0[i]
        else:
            continue
    if (Consistency(X0, Y, eligTUP) == "1"):
        eng.flush()
        print("Consistent phase",phase)
        
        prob00000 = eng.backend.get_probability('00000', X0)
        prob00001 = eng.backend.get_probability('00001', X0)
        prob00010 = eng.backend.get_probability('00010', X0)
        prob00011 = eng.backend.get_probability('00011', X0)
        prob00100 = eng.backend.get_probability('00100', X0)
        prob00101 = eng.backend.get_probability('00101', X0)
        prob00110 = eng.backend.get_probability('00110', X0)
        prob00111 = eng.backend.get_probability('00111', X0)
        prob01000 = eng.backend.get_probability('01000', X0)
        prob01001 = eng.backend.get_probability('01001', X0)
        prob01010 = eng.backend.get_probability('01010', X0)
        prob01011 = eng.backend.get_probability('01011', X0)
        prob01100 = eng.backend.get_probability('01100', X0)
        prob01101 = eng.backend.get_probability('01101', X0)
        prob01110 = eng.backend.get_probability('01110', X0)
        prob01111 = eng.backend.get_probability('01111', X0)
        prob10000 = eng.backend.get_probability('10000', X0)
        prob10001 = eng.backend.get_probability('10001', X0)
        prob10010 = eng.backend.get_probability('10010', X0)
        prob10011 = eng.backend.get_probability('10011', X0)
        prob10100 = eng.backend.get_probability('10100', X0)
        prob10101 = eng.backend.get_probability('10101', X0)
        prob10110 = eng.backend.get_probability('10110', X0)
        prob10111 = eng.backend.get_probability('10111', X0)
        prob11000 = eng.backend.get_probability('11000', X0)
        prob11001 = eng.backend.get_probability('11001', X0)
        prob11010 = eng.backend.get_probability('11010', X0)
        prob11011 = eng.backend.get_probability('11011', X0)
        prob11100 = eng.backend.get_probability('11100', X0)
        prob11101 = eng.backend.get_probability('11101', X0)
        prob11110 = eng.backend.get_probability('11110', X0)
        prob11111 = eng.backend.get_probability('11111', X0)
            
            
            
            
        print("Probability to measure C 00000: {}".format(prob00000))
        print("Probability to measure C 00001: {}".format(prob00001))
        print("Probability to measure C 00010: {}".format(prob00010))
        print("Probability to measure C 00011: {}".format(prob00011))
        print("Probability to measure C 00100: {}".format(prob00100))
        print("Probability to measure C 00101: {}".format(prob00101))
        print("Probability to measure C 00110: {}".format(prob00110))
        print("Probability to measure C 00111: {}".format(prob00111))
        print("Probability to measure C 01000: {}".format(prob01000))
        print("Probability to measure C 01001: {}".format(prob01001))
        print("Probability to measure C 01010: {}".format(prob01010))
        print("Probability to measure C 01011: {}".format(prob01011))
        print("Probability to measure C 01100: {}".format(prob01100))
        print("Probability to measure C 01101: {}".format(prob01101))
        print("Probability to measure C 01110: {}".format(prob01110))
        print("Probability to measure C 01111: {}".format(prob01111))
        print("Probability to measure C 10000: {}".format(prob10000))
        print("Probability to measure C 10001: {}".format(prob10001))
        print("Probability to measure C 10010: {}".format(prob10010))
        print("Probability to measure C 10011: {}".format(prob10011))
        print("Probability to measure C 10100: {}".format(prob10100))
        print("Probability to measure C 10101: {}".format(prob10101))
        print("Probability to measure C 10110: {}".format(prob10110))
        print("Probability to measure C 10111: {}".format(prob10111))
        print("Probability to measure C 11000: {}".format(prob11000))
        print("Probability to measure C 11001: {}".format(prob11001))
        print("Probability to measure C 11010: {}".format(prob11010))
        print("Probability to measure C 11011: {}".format(prob11011))
        print("Probability to measure C 11100: {}".format(prob11100))
        print("Probability to measure C 11101: {}".format(prob11101))
        print("Probability to measure C 11110: {}".format(prob11110))
        print("Probability to measure C 11111: {}".format(prob11111))
        
        if (phase%2 == 1):
            
            BreakSymmetry_VK(X0,X1,Y, phase,eligTUP)
        if (phase%2 == 0):
            BreakSymmetry_UK(X0,X1, Y, phase,eligTUP)
            
    new_eligp = new_elig_parties_max(X0,X0, eligTUP)        
    if (len(new_eligp) == 1):
        return(new_eligp)
    else:
        eligTUP = []
        EP = 0
        for i in range(len(X0)):
            if (i in new_eligp):
                eligTUP.append([i,1])
            else:
                eligTUP.append([i,0])
        print("eligTUP", "phase", phase, eligTUP)
        return(LE(n,eligTUP,phase-1))

In [ ]:
LE(n,eligTUP,n)

phase 5
Cons 5P
consistencyval 0
in findz_vals
eligtup[i][1] 1
eligtup ga z val calc 1
X0, i 0 0
X1, i 0 0
eligtup[i][1] 1
eligtup ga z val calc 2
X0, i 1 1
X1, i 1 1
eligtup[i][1] 1
eligtup ga z val calc 3
X0, i 0 2
X1, i 0 2
eligtup[i][1] 1
eligtup ga z val calc 4
X0, i 1 3
X1, i 1 3
eligtup[i][1] 1
eligtup ga z val calc 5
X0, i 1 4
X1, i 1 4
zval_list [[0, 0], [1, 3], [2, 0], [3, 3], [4, 3]]
new_elig_parties [1, 3, 4]
eligTUP phase 5 [[0, 0], [1, 1], [2, 0], [3, 1], [4, 1]]
phase 4
Cons 3P
eliglist [1, 3, 4]
cons val 3P 0
in findz_vals
eligtup[i][1] 0
eligtup[i][1] 1
eligtup ga z val calc 1

In [ ]:
#zval doesnt work

In [ ]:
print("hello")

In [ ]:
test = eng.allocate_qureg(2)